## Extract text from pdf file using llama.cpp
llama.cpp 

In [2]:
import os
import io
import time
import base64
import llama_cpp
from pdf2image import convert_from_path, pdfinfo_from_path
from llama_cpp import Llama
from llama_cpp.llama_chat_format import Llava15ChatHandler
from minio import Minio

print(f"Library Location: {llama_cpp.__file__}")
print("Libraries imported successfully.")

Library Location: /home/dhruv/docstream/services/ingestion-worker/venv/lib/python3.12/site-packages/llama_cpp/__init__.py
Libraries imported successfully.


In [ ]:
# Configure 
from dotenv import load_dotenv

load_dotenv()

BASE_DIR = ".."  # .. means the ingestion_worker
MODELS_DIR = os.path.join(BASE_DIR, "models")

MODEL_PATH = os.path.join(MODELS_DIR, "Qwen2-VL-2B-Instruct-Q4_K_M.gguf")
MMJPROJ_PATH = os.path.join(MODELS_DIR, "mmproj-Qwen2-VL-2B-Instruct-f16.gguf")

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model missing at: {MODEL_PATH}")

MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")
MINIO_ACCESS_KEY = os.getenv("MINIO_ACCESS_KEY")
MINIO_SECRET_KEY = os.getenv("MINIO_SECRET_KEY")
BUCKET_NAME = os.getenv("MINIO_BUCKET_NAME")

if not MINIO_ACCESS_KEY or not MINIO_ENDPOINT or not MINIO_SECRET_KEY or not BUCKET_NAME:
    raise ValueError(f"Error: Missing MINIO credientials in .env file")

# Processing Settings
BATCH_SIZE = 50       # Process 50 pages at a time
CHUNK_SIZE = 1000     # Vector DB chunk size
CHUNK_OVERLAP = 200   # Sliding window overlap

minio_client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False  # Set to True if using HTTPS (Production)
)
